<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/src/experimental-notebooks/compare-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compare Various HuggingFace Models for CDG by `SmoothedBLEU` Score

## Purpose

The purpose of this notebook is to figure out which models will work best for our CLI app. We will calculate and compare these select models by their smoothed BLEU scores and possibly look into fine-tuning them with [this dataset](https://www.dropbox.com/sh/488bq2of10r4wvw/AACs5CGIQuwtsD7j_Ls_JAORa/finetuning_dataset?dl=0&subfolder_nav_tracking=1).

## Models

These are the models we'll create inference points for and load in this notebook.
1. [SEBIS/code_trans_t5_base_code_documentation_generation_python](https://huggingface.co/SEBIS/code_trans_t5_base_code_documentation_generation_python)
2. [Salesforce/codet5-base-multi-sum](https://huggingface.co/Salesforce/codet5-base-multi-sum)
3. [google/flan-t5-small](https://huggingface.co/google/flan-t5-small)

The models listed above have been fine-tuned on the CodeSearchNet dataset across various programming languages for PL-NL sequence-to-sequence tasks.

## Install dependencies

In [6]:
%pip install -q transformers sentencepiece datasets nltk

Note: you may need to restart the kernel to use updated packages.


In [7]:
from transformers import AutoTokenizer, AutoModelWithLMHead, RobertaTokenizer, T5Tokenizer, T5ForConditionalGeneration, SummarizationPipeline
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from datasets import load_dataset

## Define evaluation metric

In [8]:
def calculateSmoothedBLEU(reference, candidate):
    """
    Calculate the Smoothed BLEU-4 score between a reference string and a candidate string.

    Args:
    reference (str): The reference (ground truth) string.
    candidate (str): The candidate (generated) string.

    Returns:
    float: The Smoothed BLEU-4 score.
    """
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    smoother = SmoothingFunction()
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoother.method1)

    return bleu_score

## Define models

### CodeTransT5

In [9]:
codeTransModel = AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_base_source_code_summarization_python_transfer_learning_finetune")
codeTransTokenizer = AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_base_source_code_summarization_python_transfer_learning_finetune", skip_special_tokens=True)

### CodeT5

In [10]:
codeT5Model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base-multi-sum')
codeT5Tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum', skip_special_tokens=True)

### FlanT5 

In [ ]:
flanT5Model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")
flanT5Tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small", skip_special_tokens=True)

c:\Users\deshi\anaconda3\envs\py39-torch\lib\site-packages\transformers\models\auto\modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
c:\Users\deshi\anaconda3\envs\py39-torch\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system us

KeyboardInterrupt: 

## Create pipelines

In [ ]:
codeTransPipeline = SummarizationPipeline(
    model=codeTransModel,
    tokenizer=codeTransTokenizer,
    device=0
)

In [ ]:
codeT5Pipeline = SummarizationPipeline(
    model=codeT5Model,
    tokenizer=codeT5Tokenizer,
    device=0
)

In [ ]:
flanPipeline = SummarizationPipeline(
    model=flanT5Model,
    tokenizer=flanT5Tokenizer,
    device=0
)